In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from baukit import TraceDict

In [2]:
model_name = "meta-llama/Llama-3.2-1B"
device = "mps"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model.eval()

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (ro

In [3]:
text = "medical students are typically"
layer_name = "model.layers.15"
inputs = tokenizer(text, return_tensors="pt").to(device)

In [4]:
subj_end_idx = inputs["input_ids"].shape[1] - 1

with TraceDict(model, [layer_name]) as ret:
    model(**inputs)
    
# Extract vector: [Batch, Seq, Hidden] -> [Hidden]
# Handle both tuple output (e.g., GPT-2) and tensor output (e.g., Qwen)
output = ret[layer_name].output
if isinstance(output, tuple):
    # Output is a tuple, extract the first element
    h = output[0][0, subj_end_idx, :].detach().cpu().numpy()
else:
    # Output is already a tensor
    h = output[0, subj_end_idx, :].detach().cpu().numpy()

In [5]:
print(f"Output type: {type(ret[layer_name].output)}")
print(f"Output shape: {ret[layer_name].output.shape if hasattr(ret[layer_name].output, 'shape') else 'N/A'}")

if isinstance(ret[layer_name].output, tuple):
    print(f"First element shape: {ret[layer_name].output[0].shape}")

Output type: <class 'torch.Tensor'>
Output shape: torch.Size([1, 5, 2048])
